In [ ]:
import cv2
import numpy as np
import pytesseract
from openai import OpenAI

Pre-Process the images

In [ ]:
def pre_process(src):
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    mask_1 = np.ones(src.shape[:2], dtype="uint8") * 255
    mask_2 = np.ones(src.shape[:2], dtype="uint8") * 255
    # noise = cv2.medianBlur(gray,3)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv2.imshow('Pre-Processed Image', thresh)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return thresh, mask_1, mask_2


Remove Shapes

In [ ]:
def ocr(masked):
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    config = ("-l eng — oem 1 — psm 3")
    text = pytesseract.image_to_string(masked, config=config)
    return text

Perform OCR

In [ ]:
def ocr(masked):
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    config = ("-l eng — oem 1 — psm 3")
    # pytessercat
    text = pytesseract.image_to_string(masked, config=config)
    return text
def remove_shapes(src, thresh, mask_1, mask_2):
    list1 = []
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE,
                                cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 400:
            approx = cv2.approxPolyDP(cnt,
                                      0.01 * cv2.arcLength(cnt, True), True)
            cv2.drawContours(mask_1, [approx], -1, 0, 5)
            if area < 50000:
                if (len(approx) == 4):
                    list1.append("square")
                    print("Square")
                else:
                    list1.append("circle")
                    print("Circle")
                cv2.drawContours(mask_2, [approx], -1, 0, 5)
                cv2.imshow('Masked Image', mask_2)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
    masked = cv2.bitwise_not(src, src, mask=mask_1)
    masked = cv2.bitwise_or(src, src, mask=mask_1)
    masked = cv2.bitwise_and(src, src, mask=mask_1)
    cv2.imshow('Masked Image', masked)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return masked

In [ ]:
def predict(text):
    pre_prompt = "Take the following two sets of strings as input, \
        which are extracted from two different flowcharts, \
        understand what the flowcharts are doing, \
        and \"only\" give a percentage of similarity between the two \
        as a response and nothing else, no explanation is required."
    client = OpenAI(
        api_key="INSERT KEY HERE")
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": pre_prompt}, {"role": "user", "content": text}],
    )
    # return response.choices[0].message.content
    return "ok"

Driver Code

In [ ]:
src1 = cv2.imread(
    r"D:\OneDrive\Repositories\Flowchart-Plagarism-Detector\Dataset\IMG-20231113-WA0047.jpg")
cv2.imshow('Original Image', src1)
cv2.waitKey(0)
cv2.destroyAllWindows()
thresh1, mask1_1, mask1_2 = pre_process(src1)
masked1 = remove_shapes(src1, thresh1, mask1_1, mask1_2)
text1 = "1.\n"+ocr(masked1)+"\n\n"

print("--------------------------------------------------------------------------------------")

src2 = cv2.imread(
    r"D:\OneDrive\Repositories\Flowchart-Plagarism-Detector\Dataset\IMG-20231113-WA0048.jpg")
cv2.imshow('Original Image', src2)
cv2.waitKey(0)
cv2.destroyAllWindows()
thresh2, mask2_1, mask2_2 = pre_process(src2)
masked2= remove_shapes(src2, thresh2, mask2_1, mask2_2)
text2 = "2. \n"+ocr(masked2)
# print(text1+text2)
print(predict(text1+text2))